# Credit assignment by role
Assigned AGO credits to all users of a specified role type.

USE AT YOUR OWN RISK

ArcGIS Online Python Tools for Schools: http://esriurl.com/Tools4Schools

Technical reference: https://developers.arcgis.com/python/guide/administering-your-gis/#Managing-ArcGIS-Online-credits

- trb, September 2017


In [ ]:
# import enviros and set vars
from arcgis.gis import GIS
from IPython.display import display
import sys


# ##### Add your parameters below ########################################
orgurl = ""   # use full URL, eg http://mySchoolName.maps.arcgis.com
adminuser = "" # login for the admin account creating the new users
adminpw = ""   # password for the admin account creating new users


print(" ********** ")

In [ ]:
#### source = GIS(orgurl, adminuser, adminpw)
source = GIS(orgurl, adminuser, adminpw)
# all org members
source_users = source.users.search('', max_users=1000)
print(" ********** - Organization details")
print("Total members in organization:", len(source_users))

# ## create a dictionary to hold custom roleid and role name. We use this later to delete.
list_roleIds = [] #initialize
dict_customroles = {}

for g in source_users:
    if hasattr(g, "roleId"):
        list_roleIds.append(g.roleId)
        customroles = source.users.roles.get_role(g.roleId)
        if customroles.name not in dict_customroles:
             dict_customroles[customroles.name] = g.roleId

#list all groups (built-in and custom)
# Query CUSTOM roles and add to list
roleSearch = source.users.roles.all(max_roles=100)
print('Total non-admin role types in organization, built-in and custom:', len(roleSearch)+2)
print(" ")

# ################################################
print(" ********** - Organization role selection")

# Create a data dictionary: assigned number, role name
list_orgroles = ["org_user","org_publisher"]
dict_customroles

# add cutom roles to the list_orgroles list
for k in roleSearch:
    list_orgroles.append(k.name)
    count=0
for m in list_orgroles:
    print(count,"=",m)
    count=count+1
    
# ##################################################

print('Enter a role number to assign credits to all organization members with that role type:')
ans = input()
#print(ans)
print(" ********** ")

In [ ]:
# confirm request 

#user input out of range or unconfirmed?
if (int(ans) >= 0) & (int(ans) < len(list_orgroles)):
        print ('The value you have entered is valid. Press Y to proceed or any other key to cancel.')
        confirm = input()
        if confirm != "Y":
            print ('Confirmation failed. Script ending.')
            raise KeyboardInterrupt               
else:
        print ('Entered value is out of range. Script ending.')
        raise KeyboardInterrupt     



In [ ]:
#print(list_orgroles[int(ans)])  #int forces the ans value to process before being used in list
role = list_orgroles[int(ans)]
print("Resetting members in role: " + role)
#print(ans)

# ##### Delete users #######################################    
# the path below is determined by a built-in versus custom role ###

# if BUILT-IN role (#0 or #1) then ##########################
if (ans == "0") or (ans == "1"):
    print("BUILT-IN role type")
    source_users = source.users.search("role:" + role)
    #for user in source_users:
        #print(user.username + "\t:\t" + str(user.role))
    for user in source_users:
        try:
            user = source.users.get(user.username)
            if (user is not None):
                print('Setting credits for user: ' + user.fullName + ", " + user.role)
                source.admin.credits.allocate(username=user.username, credits=1)
        except:
            print('User {} cannot be deleted or does not exist in Target Org'.format(user.username))

# if CUSTOM role then do ####################################
if int(ans) >= 2:
    print("CUSTOM role type")
  
    # IF NO USERS IN SELECTED ROLE< THIS BLOCK WILL FAIL
    try:
        searched_role = source.users.roles.get_role(dict_customroles[role])
            #print("Role name:",searched_role.name)

        for user2 in source_users:
            if hasattr(user2, 'roleId'):
                if user2.roleId==dict_customroles[role]:
                    print ('Setting credits for user: ' + user2.username)
                    source.admin.credits.allocate(username=user2.username, credits=1)
    except:
        print ("Oops!  It looks like you selected a role with no users.")
  

# ########## END OF DELETING ################################    
print("  ")
print(" ********** - Script complete. ")
